In [1]:
import math
import re
import heapq

import pandas as pd
import numpy as np

# Read line by line, remove any special characters but white space

In [2]:
test = "\\+++|||\n\t104 Nash,,, Hall, Departm$$$ent of ###Fisheri##es and Wildlife; Oregon State University, Corvallis, OR 97331-7501."
re.sub('[^A-Za-z0-9 ]+', '', test)
# re.split(", |; | ", t)
# ''.join(e for e in t if e.isalnum())

'104 Nash Hall Department of Fisheries and Wildlife Oregon State University Corvallis OR 973317501'

In [150]:
test = "./data/final_orgs.txt"
samples = 1000
query = []
with open(test) as f:
    while samples > 0:
        query.append(re.sub('[^A-Za-z0-9 ]+', '', (next(f))))
        samples -= 1

# Remove empty string
query = query[62:]

In [151]:
train = "./data/academics.csv"
df = pd.read_csv(train)
data = df.Features.tolist()
data[:5]

['Community College of the Air Force 130 W Maxwell Blvd Montgomery AL 361126613',
 'Community College of the Air Force 2250 Stanley Road Unit 161 Fort Sam Houston TX 782346400',
 'Alabama A  M University 4900 Meridian Street Normal AL 35762',
 'University of Alabama at Birmingham 1720 2nd Avenue South Birmingham AL 35233',
 'UAB School of Optometry 1716 University Boulevard Birmingham AL 35294']

In [18]:
df.sample(20)

,University,Features
19858,Middlebury College,Middlebury College at Bread Loaf School of Eng...
19156,Tennessee College of Applied Technology Ripley,Bells Instructional Service Center 6514 Highwa...
11079,Ross Medical Education Center,Ross Medical Education Center Charleston 5707...
26012,MarksTo Beauty Academy,MarksTo Beauty Academy 1614 Brentwood Drive Pi...
16740,Wright State University,Wright State University at Washington Court Ho...
10476,Central Michigan University,Central Michigan University at Offutt AFB Cent...
22461,Northwest Technology CenterFairview,Northwest Technology CenterFairview 801 Tech D...
4081,American Health Information Management Associa...,American Health Information Management Associa...
13365,Bellevue University,Bellevue University at Western Iowa Technical ...
26037,American English College,American English College 111 N Atlantic Blvd S...


# Compute the Tf-Idf table

In [46]:
import numpy.linalg as LA

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [179]:
vectorizer = CountVectorizer()
transformer = TfidfTransformer()

dummy = data
# dummy_test = data[20:30]
dummy_test = query[100:120]

# Shape: (5, 42)
train = vectorizer.fit_transform(dummy).toarray()

# Shape: (2, 42)
test = vectorizer.transform(dummy_test).toarray()

scores = np.zeros(shape=(len(train), len(test)))

## Solution1: For-loop
# for i, vector in enumerate(train):
#     for j, testv in enumerate(test):
#         score = np.inner(vector, testv)/(LA.norm(vector)*LA.norm(testv))
#         scores[i][j] = score

## Solution2: Vectorized
denominator = np.dot(LA.norm(train, axis=1).reshape(-1, 1),
                     LA.norm(test, axis=1).reshape(1, -1))
scores_array = np.divide(np.dot(train, test.T), denominator)
groups = scores_array.argmax(axis=0)

/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:25: RuntimeWarning: invalid value encountered in true_divide


In [180]:
for i, group in enumerate(groups):
    print("String:\t{}".format(dummy_test[i][-70:]))
    print("Group:\t{}".format(dummy[group][:70]))
    print("Similarity:\t{}".format(scores_array[group][i]))
    print()

String:	0414
Group:	Community College of the Air Force 130 W Maxwell Blvd Montgomery AL 36
Similarity:	nan

String:	Dep of Plant and Soil Sciences Oklahoma State Univ Stillwater OK 74078
Group:	Oklahoma State University 107 Whitehurst Hall Stillwater OK 74078
Similarity:	0.5547001962252291

String:	044 N Agricultural Hall Oklahoma State University Stillwater OK 74078
Group:	Oklahoma State University 107 Whitehurst Hall Stillwater OK 74078
Similarity:	0.8249579113843053

String:	04UR1502
Group:	Community College of the Air Force 130 W Maxwell Blvd Montgomery AL 36
Similarity:	nan

String:	04UR150204UR1502
Group:	Community College of the Air Force 130 W Maxwell Blvd Montgomery AL 36
Similarity:	nan

String:	11 Shijiazhuang Department of Molecular Biology Hebei Cancer Institute
Group:	Illinois Institute of Technology  Institute of Design 350 N LaSalle Ch
Similarity:	0.4472135954999579

String:	051 Base CNGC Huayin China
Group:	Colorado State University  Pueblo at Peterson Air Force Base P

# Query

In [104]:
# heapq.nlargest(10, a.query("University"))
heapq.nlargest(10, a.query("Georgia Institute of Technology"))
# heapq.nlargest(10, a.query("100 Radiation Center Oregon State University Corvallis"))

[(0.39864781862538884, '10524 California Institute of Technology'),
 (0.3444626099831442,
  '0ffice of Information Teclhnology Georgia Institute of TechnologyTAB'),
 (0.23503090133769758,
  '10524 Robinson Lab California Institute of Technology Pasadena California 91125'),
 (0.09859764475914322,
  '0601 class of Computer Science  Technology Hubei University of Technology'),
 (0.08551410564996555,
  '100 Haiquan Road Dept of Food Science and Technology Shanghai Inst of Technology Shanghai China'),
 (0.08468069944196469,
  '100 Bureau DR National Institute of Standards and Technology Gaithersburg MD 208998662 USA'),
 (0.07763169629565018,
  '100 Bureau Drive Stop 8940 National Institute of Standards  Technology Gaithersburg Maryland 20899TAB'),
 (0.0773660503472803,
  '100 Bureau Drive Stop 8423 National Institute of Standards and Technology Gaithersburg Maryland 208998423 USA'),
 (0.07322102476361969, '101st University Hospital Clinic Tbilisi Georgia'),
 (0.07061979351151958,
  '100 Bur